<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

In [0]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

In [0]:
import pandas as pd
df = pd.read_csv("data/df_imputed_week_shift.csv", index_col = 0, parse_dates = True)